In [1]:
import numpy as np
import pandas as pd
import warnings
import json
import requests
import random
from elasticsearch import Elasticsearch
warnings.filterwarnings('ignore')

# Exploračná analýza

## Tabuľka events

In [2]:
events = pd.read_csv('vi_data/events_train.csv')
events.sample(10)

,customer_id,timestamp,event_type,product_id,title,category_name,price
10704929,8f702e1e-c046-4c91-8b63-4b97aff10a8d,2019-08-17 06:10:20.660117 UTC,view_item,c09c1152-a163-4d00-b11a-d8ad08ea396a,Women's skirt SAM73 GAILIA,Women>Women's Clothing>Skirts>Mini skirts,7.53
5299206,8b8423f2-9284-4db8-9e73-4945656c572a,2019-07-26 18:08:28.006863 UTC,view_item,47cda0bb-3322-420e-bd5e-7914cbe60a7e,Made Of Emotion Woman's Jumpsuit M382 Powder,Women>Women's Clothing>Jumpsuits,40.08
7708020,84131674-5722-4a18-a322-d025c0a7ce87,2019-08-05 14:01:34.026415 UTC,view_item,5c57095d-76a9-4a6f-8f47-3a0f97d2dfea,Adidas Nemeziz 17.3 FG Mens Football Boots,Men>Men Footwear>Football Boots & Indoor Trainers,45.70
8299183,0c962945-bc92-4089-8678-c78d9d04f243,2019-08-07 16:24:13.193066 UTC,view_item,94bf2067-89fb-4c4e-8a82-3a76dc67a4b3,Figl Woman's Dress M660,Women>Women's Clothing>Dresses>Midi dress,39.41
3182710,f55e5d7b-3e16-41cd-95ae-ecd01eb27a8f,2019-07-16 05:49:03.177559 UTC,view_item,322bb79d-52d8-49e3-8078-7485bf615d71,No Fear Double Waist Shorts Mens,NaN,21.90
4959200,58587004-a7d2-4e1f-a3b9-f0fcb751bc97,2019-07-24 23:29:15.939124 UTC,view_item,1b07b6db-1e6f-422d-87df-8918239247e2,Made Of Emotion Woman's Dress M296,Women>Women's Clothing>Dresses>Midi dress,39.01
7990197,c9442631-df03-49be-8448-798adc112fa4,2019-08-06 14:15:19.030003 UTC,view_item,01b2086b-1ef1-4b32-8b51-1f5698fc7b6c,Adidas TERREX AX2R Mid Junior Walking Boots,Children>Children's Footwear>Hiking shoes>High...,76.02
7578106,7b2e3f19-9866-4943-adcb-82dc2600b4e8,2019-08-05 05:08:20.252862 UTC,view_item,1da7e9e9-ee3a-4812-a27d-1efcc95c4006,New Balance M520v5 Trainers Mens,Sports>Running>Running Shoes>Men's Running Shoes,63.37
9999635,c39bd5f6-52c3-48d0-936e-fbf7fbd72aed,2019-08-14 10:10:00.485133 UTC,view_item,ced5b9a4-e168-4c47-b1c7-8428880d2fd6,Lee Cooper Knit Jacket Mens,Men>Men Clothing>Sweaters>Zip-up sweaters,18.37
4812034,4180f9ee-4854-4d69-a1f2-29fa29544932,2019-07-24 09:05:32.009108 UTC,view_item,11c70685-e0dd-423e-8029-b3977e582ce8,Biba Icon swimsuit with tummy control dámske,NaN,92.13


In [4]:
events.shape

(14614385, 7)

In [5]:
events.event_type.unique()

array(['view_item'], dtype=object)

## Tabuľka purchases

In [3]:
purchases = pd.read_csv('vi_data/purchases_train.csv')
purchases.sample(10) 

,customer_id,timestamp,event_type,product_id,title,category_name,price
97576,f103cb77-74a1-4321-9c95-0080b876dce1,2019-08-04 14:20:01.114696 UTC,purchase_item,b572f0d3-5d63-4695-808f-932969ba1bdb,Kids socks MANY MORNINGS,Children>Girl's Clothing>Underwear>Socks,3.51
97947,39843a42-b9f2-4858-b000-b960fa9ec739,2019-08-04 16:22:00.629292 UTC,purchase_item,6e82ee55-6a61-48f0-ae26-bf73ebce4d86,Womens hoodie Fruit of the Loom Basic,Women>Women's Clothing>Sweatshirts>Hoodies,9.01
65238,1ae583a2-5730-4ff6-afa6-d76ee7757b7a,2019-07-25 11:27:55.507235 UTC,purchase_item,023ed07b-b1f4-4b75-be0d-56330528a799,Muddyfox Cycle Jacket Mens,Men>Men Clothing>Jackets & Coats>Lightweight s...,26.49
54762,a3c411c0-f31a-4f4d-9140-f1a766ae3b46,2019-07-20 20:24:40.817386 UTC,purchase_item,c0bc3023-a401-4790-ad6c-7e58ff8fb975,Trendyol Orange high waist Bikini bottom,Women>Women's Clothing>Swimmsuits & bikins,11.58
124888,6c6fb741-1d37-4570-ae63-5e758875a5d2,2019-08-12 09:38:44.866307 UTC,purchase_item,73608106-72cd-4370-b6d4-a3bfd599b7c0,Nike Tanjun Girls Trainers,Women>Women's Footwear>Sneakers>Sport sneakers,52.43
75922,49e43e9f-a8b0-44bc-b5c8-3e70df3cfb4d,2019-07-29 08:31:00.299825 UTC,purchase_item,d94e8a8d-64e0-4fca-953f-9b762f49a7db,Women's Dress NUMOCO 59,Women>Women's Clothing>Dresses>Mini dress,29.40
90685,19dd5075-3d49-4a87-9011-a0a51fb58a61,2019-08-02 17:14:04.209189 UTC,purchase_item,c29fe27a-2146-4d77-8163-b0f6294e0354,Trendyol Khaki Swimsuit,Women>Women's Clothing>Swimmsuits & bikins>One...,28.56
178054,8d67f4be-60e5-44a8-a225-0c29e1c4d284,2019-08-28 19:27:07.030909 UTC,purchase_item,b8e230bc-dbb1-492b-9799-9eb9582c40ba,Slazenger 3 Pack Briefs Mens,Men>Men Clothing>Underwear>Briefs,9.96
126916,1809e145-1096-456a-b0b0-2fc99d3438b9,2019-08-12 18:55:47.886959 UTC,purchase_item,a2e6be59-8f01-4543-b536-09403038d415,Women's Briefs Lee Cooper 2 Pack,Women>Women's Clothing>Underwear>Panties,4.41
120498,6b836d2b-3c46-4db7-8c38-d7550330b57c,2019-08-10 22:07:49.137156 UTC,purchase_item,3973d445-ba2a-4eb4-8bc1-b227a1faa9ea,Lee Cooper Boxers 5 Pack,Men>Men Clothing>Underwear,11.52


In [7]:
purchases.shape

(188712, 7)

In [8]:
purchases.event_type.unique()

array(['purchase_item'], dtype=object)

tabulky events a purchaches majú roznaké stĺpce 

## Spojenie events and purchases

In [4]:
data = pd.concat([events, purchases], ignore_index=True)
data.sample(10)

,customer_id,timestamp,event_type,product_id,title,category_name,price
7432605,6abcc5fa-ad10-48e2-a5a5-443d98e32138,2019-08-04 14:53:48.692227 UTC,view_item,2d3d0e2c-112f-468f-878b-694dfbcf501b,Firetrap LDN Bum Bag 93,Accessories >Backpacks and Bags>Bum bags,20.45
9645703,8d8feb23-b700-47e6-8509-34a3f8413a10,2019-08-12 20:01:36.577007 UTC,view_item,15b3e401-6159-46e2-94dd-1456cd144597,Puma Court Breaker Mens Trainers,Men>Men Footwear>Sneakers>Low sneakers,50.62
12831915,1ceedea3-9d2f-41ea-ab06-410e157ce2bb,2019-08-25 17:17:28.629495 UTC,view_item,7093f50c-c7d9-4015-ab6e-bc832b9d44ff,Lemoniade Woman's Sweater LS213,Women>Women's Clothing>Sweaters & Cardigans>Ca...,34.12
2380034,69080ffe-407a-44ee-91bc-e82d4e9d6e5d,2019-07-12 16:14:09.839359 UTC,view_item,a1547fd5-c6c0-4dc0-989d-5fe286c893d6,DC Blitz II Trainers Mens,NaN,73.69
7734169,817d8add-1e27-4db9-94d2-6624e89e0e15,2019-08-05 15:48:34.34332 UTC,view_item,827bda66-2718-4549-8d56-39bfd1772a5b,Pierre Cardin Stripe Swimshorts Mens,Men>Men Clothing>Shorts,13.15
11914618,93234677-a0ae-423c-bef6-a05532f0f24e,2019-08-22 05:54:33.776936 UTC,view_item,f43e7f41-691f-4474-939c-a7487d82ce3c,Crafted Junior Boys 3 Pack Jog Set,Children>Boy's Clothing>Boy's sets,20.14
8511242,222791a6-09c0-497e-8b7a-e05a19e95877,2019-08-08 11:41:39.312886 UTC,view_item,b5861be8-fec3-4058-959d-f47f015f1598,Prince Reflex Mens Tennis Shoes,Men>Men Footwear>Sneakers>Sport sneakers,57.42
6251879,464b63ec-0142-4a72-8cd9-fb0b7f0132d2,2019-07-30 20:04:46.710069 UTC,view_item,fa7b221d-95a1-48a3-96b2-c19a2bc6e7dc,Trendyol Black Patent Leather Detailed Women's...,Women>Women's Footwear>Court shoes,25.26
4502318,46d04df5-6331-4e61-b907-3f29f00b133c,2019-07-22 17:54:34.156284 UTC,view_item,6aa5329b-719e-4087-aaad-8f24a9ce24c0,Slazenger Canvas Kung Snr93,Men>Men Footwear>Sneakers>Low sneakers,16.97
9041043,1ad4cf0a-c156-4888-924d-88d5cf39d788,2019-08-10 14:08:07.641959 UTC,view_item,d5a8b3f3-da4e-4009-89c6-f8102a256a8c,Blowfish Balta Boots,Women>Women's Footwear>Ankle shoes>on wedge heel,38.14


In [25]:
rows_num, cols_num = data.shape
print(rows_num)
print(cols_num)

14803097
7


In [5]:
del events
del purchases

## Atribúty

### customer_id

In [11]:
data.customer_id.describe()

count                                 14803097
unique                                 2211386
top       53f841c7-970b-4e0b-96bf-97a95ecfb734
freq                                      4804
Name: customer_id, dtype: object

In [12]:
len(data[data.customer_id.isna()])

0

In [13]:
rows_num/len(data.customer_id.unique())

6.694035776657716

In [14]:
data.groupby(['customer_id']).count()['timestamp'].median()

2.0

In [14]:
customer_rows = data['customer_id'].value_counts()
customer_rows

53f841c7-970b-4e0b-96bf-97a95ecfb734    4804
9eb76d09-8006-4262-b5eb-459556f338a9    3577
546a1146-64be-4549-a472-a8a4a8ca41b0    3189
a3fe2d55-b8fc-4aaa-8f7f-99f6498b191f    3060
68b2c520-5366-4143-a253-04fd96aeb251    2855
                                        ... 
28f3de37-5d91-451b-9d71-c016ff28765a       1
ddaea6ea-cf1e-4ef3-84d0-4d97f5019dab       1
ee2c4b69-2194-4785-a817-76a30ea8a54b       1
db9f09e8-ea9e-4444-986a-39c973a7dfe3       1
3fe6ea93-4d29-4d6f-990a-6a873f19c2ed       1
Name: customer_id, Length: 2211386, dtype: int64

In [28]:
for i, num in enumerate(customer_rows.value_counts()[:3]):
    print(i+1, num)

1 913643
2 329587
3 199175


In [29]:
(913643)/2211386

0.4131540129131685

In [20]:
(913643+329587)/2211386

0.5621949311427313

In [26]:
for i, num in enumerate(customer_rows.value_counts()[3:50:4]):
    print(4*i+4, num)

4 129721
8 45301
12 22832
16 13311
20 8728
24 6193
28 4481
32 3349
36 2677
40 2124
44 1784
48 1405


- 2,211,386 unikátnych zákazníkov
- žiadne NaN hodnoty
- v priemere vyše 6 záznamov na zákazníka
    - ale median je iba 2
    - 913,643 (41%) zákazníkov má len 1 záznam
    - 56% zákazníkov má 2 záznamy

### timestamp

In [16]:
data.timestamp.describe()

count                           14803097
unique                          14802889
top       2019-07-22 04:33:52.889148 UTC
freq                                   4
Name: timestamp, dtype: object

In [17]:
data.loc[data['timestamp'] == "2019-07-22 04:33:52.889148 UTC"]

,customer_id,timestamp,event_type,product_id,title,category_name,price
4361630,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,5bbe88b4-9650-4c1a-8f95-9a09c264b2c9,Hot Tuna Crew T Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts,12.84
4361631,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,248faf65-d1e2-4d7b-84c8-ce1c459ee086,Lonsdale Jersey Polo Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts>Polo S...,12.90
4361632,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,377bdb75-6f46-4c29-b559-9af0c6ba2ae8,Pierre Cardin Web Belt Shorts Mens,Men>Men Clothing>Shorts,17.12
4361633,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,b2bcef2f-df2d-4c23-aee7-73ca112ef815,Lee Cooper Men's boxers 5-pack,Men>Men Clothing>Underwear>Boxer shorts & trunks,13.10


In [47]:
# data["timestamp"] = pd.to_datetime(data["timestamp"])

In [18]:
min(data.timestamp)

'2019-07-01 00:00:01.101977 UTC'

In [19]:
max(data.timestamp)

'2019-08-31 23:59:59.536067 UTC'

In [7]:
len(data[data.timestamp.isna()])

0

- dátumy z intervalu 2 mesiacov
- žiadne NaN hodnoty
- existujú záznamy typu `view_item`, kde je daný zákazník v danom čase a rôzne produkty. 

### event_type

In [21]:
data.event_type.unique()

array(['view_item', 'purchase_item'], dtype=object)

In [22]:
len(data[data.event_type.isna()])

0

In [23]:
len(data.loc[data['event_type'] == "view_item"])/rows_num

0.9872518568242848

In [24]:
len(data.loc[data['event_type'] == "purchase_item"])/rows_num

0.01274814317571519

- dva typy: `view_item` a `purchase_item`
- pomer je 98.7% ku 1.27%
- žiadne NaN hodnoty

### product_id

In [25]:
data.product_id.describe()

count                                 14803097
unique                                  123837
top       31ae211d-57a9-4bfa-b95f-40776af03c3f
freq                                     52736
Name: product_id, dtype: object

In [60]:
len(data[data.product_id.isna()])

0

In [61]:
rows_num/len(data.product_id.unique())

119.53694776197744

In [62]:
data.groupby(['product_id']).count()['timestamp'].median() 

17.0

In [32]:
product_rows = data['product_id'].value_counts()
product_rows

31ae211d-57a9-4bfa-b95f-40776af03c3f    52736
0f943312-7141-4606-abfa-81fd63a5498f    49141
a8f0292d-5fca-42b9-b0d3-b38e7efa416b    48343
e33cea18-ee3f-4e37-83a7-1d88c54d83b8    45421
608f32cd-210c-4af5-99d8-6182a9678cf5    39801
                                        ...  
6b0e71d0-cdca-4160-8df1-ae7b43ef967c        1
630d92d1-ed2b-4a75-a778-3dacba195eb5        1
0633f706-85d4-4ac6-b34d-7f653fa7139d        1
a3c46419-d8fb-4273-88ba-06dc92b42696        1
326e712f-9733-4761-b458-217f3f695833        1
Name: product_id, Length: 123837, dtype: int64

In [36]:
for i, num in enumerate(product_rows.value_counts()[:50:4]):
    print(4*i+1, num)

1 15982
5 3582
9 2365
13 1792
17 1376
21 1139
25 948
29 810
33 694
37 624
41 528
45 501
49 436


- 123,837 unikátnych produktov
- žiadne NaN hodnoty
- v priemere vyše 119 záznamov na produkt
    - maximum je 52,736
    - minimum je 1
    - median je 17
    - 15,982 produktov má len 1 záznam

### title

In [63]:
data.title.describe()

count                     14801284
unique                       71440
top       Lee Cooper Marl Backpack
freq                        168766
Name: title, dtype: object

In [72]:
data.loc[data['product_id'] == "31ae211d-57a9-4bfa-b95f-40776af03c3f"].title.unique()

array(['Batoh Lee Cooper Marl', 'Lee Cooper Marl Backpack'], dtype=object)

In [64]:
len(data[data.product_id.isna()])

0

- 71,440 unikátnych označení (našli sme 123,837 unikátnych produktov)
    - produkt môže mať viac rôznych označení
- žiadne NaN hodnoty

### price

In [85]:
data.price.describe().apply(lambda x: format(x, 'f'))

count    14758211.000000
mean           33.122065
std           121.532833
min             0.060000
25%            12.710000
50%            21.860000
75%            39.490000
max         32636.460000
Name: price, dtype: object

In [87]:
np.nanmedian(data.price)

21.86

In [73]:
len(data[data.price.isna()])

44886

In [89]:
data.loc[data['product_id'] == "31ae211d-57a9-4bfa-b95f-40776af03c3f"].price.unique()

array([12.9 , 12.73, 13.11, 13.32, 12.49, 12.71, 13.45, 12.65, 12.83,
       11.46, 11.8 , 11.44, 11.38, 12.1 , 11.24, 11.61, 11.99, 11.55,
        9.55,  9.56, 10.61, 10.63, 13.49,  7.32, 12.14, 12.27, 12.33,
       13.04, 12.92, 12.51, 12.11, 12.45, 12.35, 13.09, 16.84, 16.41,
       16.11, 16.03, 16.23, 15.95, 16.  , 15.58, 16.39, 16.97, 13.65,
       14.75, 13.98, 14.02, 14.05, 14.22, 14.38, 14.87, 14.12, 14.37,
       12.46, 12.37, 13.12, 12.04, 13.02, 12.39, 12.68, 12.55, 12.69,
         nan])

- priemer ceny je cca 33
- median ceny je cca 22
- 44,886 NaN hodnôt
- jeden produkt môže mať viac cien

### category_name

In [90]:
data.category_name.describe()

count                                   11367365
unique                                       713
top       Men>Men Footwear>Sneakers>Low sneakers
freq                                      462179
Name: category_name, dtype: object

In [103]:
for pid in data.product_id.sample(10):
    unique_categories = data.loc[data['product_id'] == pid].category_name.unique()
    print(len(unique_categories), unique_categories)

2 [nan 'Men>Men Clothing>T-Shirts & Polo shirts>Polo Shirts']
3 [nan "Women>Women's Clothing>Shirts"
 "Women>Women's Clothing>Shirts>Patterned shirts"]
2 [nan "Women>Women's Clothing>Jackets & Coats>Coats"]
2 [nan "Women>Women's Footwear>Slip-on"]
2 [nan "Women>Women's Clothing>Dresses>Wedding dress"]
2 [nan 'Men>Men Clothing>T-Shirts & Polo shirts>Polo Shirts']
2 [nan "Sports>Running>Running Shoes>Women's Running Shoes"]
2 [nan 'Accessories >Backpacks and Bags>Fashion backpacks']
3 ["Women>Women's Clothing>Pants & Jeans>Pants"
 "Women>Women's Clothing>Pants >Pants"
 "Women>Women's Clothing>Pants >Single-colored pants"]
2 [nan "Women>Women's Clothing>Jumpsuits"]


In [96]:
category_name_nans_num = len(data[data.category_name.isna()])
category_name_nans_num

3435732

In [95]:
category_name_nans_num/rows_num

0.23209548650529008

In [45]:
data["category_name"].str.split(">", expand = True).shape

(14803097, 7)

In [40]:
data_15K = data.sample(15000)

In [116]:
categories = data_15K["category_name"].str.split(">", expand = True)

Keďže `data` má skoro 15M záznamov, spracovanie v niektorkých prípadoch trvá veľmi dlho. Vybrali sme len 15K.

In [123]:
categories.sample(5)

,0,1,2,3,4
2563389,NaN,NaN,NaN,NaN,NaN
7189282,Women,Women's Footwear,Balerinas,None,None
11478593,Men,Men Clothing,T-Shirts & Polo shirts,Short Sleeve,None
6003198,Women,Women's Clothing,Swimmsuits & bikins,One-piece,None
1495798,NaN,None,None,None,None


In [136]:
for i in range(5):
    print('col_{} unique values ='.format(i), len(categories[i].unique()))
    print('col_{} NaN values ='.format(i), len(categories[categories[i].isna()]))
    print()

col_0 unique values = 21
col_0 NaN values = 3455

col_1 unique values = 38
col_1 NaN values = 3551

col_2 unique values = 154
col_2 NaN values = 3669

col_3 unique values = 216
col_3 NaN values = 6845

col_4 unique values = 49
col_4 NaN values = 14513



In [126]:
categories[0].unique()

array([nan, 'Men', 'Women', 'Sports', "Women's Clothing", 'Children',
       'Accessories ', 'Backpacks and Bags', 'Men Clothing',
       "Boy's Clothing", 'Sale - Men', 'Sale - Women', 'Dresses',
       'NEW IN', 'Unlisted', "Children's Footwear", 'strom kategorii',
       'T-Shirts', 'NY FASHION WEEK', 'Sale - Children',
       'Delivery Immediately'], dtype=object)

- **iba** 713 unikátnych kategórií (našli sme 123,837 unikátnych produktov)
- jeden produkt môže mať rôzne `category_name` hodnoty, vrátane NaN
- 23.2% záznamov je NaN

- po rozparsovaní stromu `category_name` vzniklo 7 stĺpcov (t.j. výška stromu)
- smerom od koreňa rastie počet NaN hodnôt a unikátnych hodnôt
- vyskytujú sa tématicky rovnaké len inak napísané kategórie


### Diskusia
Potreba vyriešiť nejednotné atribúty rovnakého produktu:
- `title`
- `category_name`
- možno `cena`

---

# Predspracovanie a oprava dát

In [7]:
categories = data["category_name"].str.split(">", expand = True)
del data["category_name"]
category_names = {}
for i in range(7):
    category_names[i] = "category_{}".format(i) 
categories = categories.rename(columns=category_names)
data = pd.concat([data, categories], axis=1, sort=False)
del categories

Vyparsovanie `category`

In [2]:
# data.to_csv('vi_data/data_train.csv', encoding='utf-8')
data = pd.read_csv('vi_data/data_train.csv', index_col=0)

Uloženie medzi-výsledku

In [219]:
data.sample(3)

,customer_id,timestamp,event_type,product_id,title,price,category_0,category_1,category_2,category_3,category_4,category_5,category_6
13029308,a307441c-2131-4cfe-a3f5-ceba85324cdb,2019-08-26 11:38:54.520998 UTC,view_item,7e40885a-c1e4-48e3-80e3-71951fc25a44,Children's Ski jacket LOAP FALDA,39.52,Children,Boy's Clothing,Jackets,Winter jackets,NaN,NaN,NaN
9634041,0e42ec0e-8d82-4d4a-b6ef-17ef06e3de1a,2019-08-12 19:25:21.494362 UTC,view_item,43bee0d7-2d3f-42ac-bef2-99de2b660be2,Top Secret LADY'S DRESS,15.34,Women,Women's Clothing,Dresses,Mini dress,NaN,NaN,NaN
14697110,7c04bdf3-0ea1-4830-9047-0350d7071ef2,2019-07-31 09:10:17.624419 UTC,purchase_item,ca3ccb9a-933f-45fb-a10a-dfde4eb046ee,Everlast Heavy Bomber Jacket Mens,35.00,Men Clothing,Men,Jackets & Coats,Bomber jackets,NaN,NaN,NaN


# Vytvorenie elastic indexu a naplnenie produktami

**PUT http://localhost:9200/vi_products**
```json
{'settings': {'number_of_shards': 1, 'number_of_replicas': 0},
 'mappings': {'properties': {'product_id': {'type': 'keyword'},
   'title': {'type': 'text', 'analyzer': 'english'},
   'price': {'type': 'double'},
   'events_number': {'type': 'integer'},
   'category_0': {'type': 'keyword'},
   'category_1': {'type': 'keyword'},
   'category_2': {'type': 'keyword'},
   'category_3': {'type': 'keyword'},
   'category_4': {'type': 'keyword'},
   'category_5': {'type': 'keyword'},
   'category_6': {'type': 'keyword'}}}}
```

In [194]:
product_ids = list(data.product_id.unique())

In [195]:
len(product_ids)

123837

In [192]:
def fill_index():
    es = Elasticsearch([{'host': '127.0.0.1', 'port': 9200}])
    doc = {}
    for pid in product_ids:
        rows = data[data.product_id == pid]
        doc['product_id'] = pid
        try:
            doc['title'] = list(rows.title.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['price'] = np.mean(rows.price.dropna().unique())
        except IndexError:
            pass
        try:
            doc['events_number'] = len(rows)
        except IndexError:
            pass
        try:
            doc['category_0'] = list(rows.category_0.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_1'] = list(rows.category_1.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_2'] = list(rows.category_2.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_3'] = list(rows.category_3.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_4'] = list(rows.category_4.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_5'] = list(rows.category_5.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_6'] = list(rows.category_6.dropna().unique())[0]
        except IndexError:
            pass
        if np.isnan(doc['price']):
            del doc['price']
        es.index(index='vi_products', body=doc)

In [218]:
# fill_index()

# Obsahový odporúčač

In [194]:
def cr2(product_ids, count=10):
    url = 'http://localhost:9200/vi_products/_search'
    def transform_id(pid):
        data = {'query': {'bool': {'filter': {'term': {'product_id': pid}}}}}
        r = requests.post(url, json=data)
        try:
            return json.loads(r.text)['hits']['hits'][0]['_id']
        except IndexError:
            return None
    
    ids = [transform_id(pid) for pid in product_ids if pid]
    should = []
    for i in ids:
        if i:
            mlt = {
              "more_like_this": {
                "fields": [
                  "title", "category_0", "category_1", "category_2", "category_3"
                ],
                "like": [
                  {
                    "_index": "vi_products",
                    "_id": i
                  }
                ],
                "min_term_freq": 1,
                "min_doc_freq": 1,
                "max_query_terms": 100,
                "include": True
              }
            }
            should.append(mlt)
            
    data = {
              "from": 0,
              "size": 10,
              "query": {
                "bool": {
                  "should": should
                }
              }
            }    
#     print(json.dumps(data))
    r = requests.post(url, json=data)
    try: 
        return [hit['_source']['product_id'] for hit in json.loads(r.text)['hits']['hits']]
    except KeyError as e:
        print('KeyError:'+e, r.text) 

In [195]:
def cr3(product_ids, count=10):
    url = 'http://localhost:9200/vi_products/_search'
    def transform_id(pid):
        data = {'query': {'bool': {'filter': {'term': {'product_id': pid}}}}}
        r = requests.post(url, json=data)
        try:
            return json.loads(r.text)['hits']['hits'][0]['_id']
        except IndexError:
            return None
    
    ids = [transform_id(pid) for pid in product_ids if pid]
    queries = []
    for i in ids:
        if i:
            mlt = {
              "more_like_this": {
                "fields": [
                  "title", "category_0", "category_1", "category_2", "category_3"
                ],
                "like": [
                  {
                    "_index": "vi_products",
                    "_id": i
                  }
                ],
                "min_term_freq": 1,
                "min_doc_freq": 1,
                "max_query_terms": 100,
                "include": True
              }
            }
            queries.append(mlt)
            
    data = {
                "from":0,
                "size":10,
                "query": { 
                    "dis_max": {
                       "queries": queries
                    }
                }
            }   
#     print(json.dumps(data))
    r = requests.post(url, json=data)
    try: 
        return [hit['_source']['product_id'] for hit in json.loads(r.text)['hits']['hits']]
    except KeyError as e:
        print('KeyError:'+e, r.text) 

# Evaluácia

## Metriky

In [204]:
def precision_at_k(recommendation, test, k=10):
    relevance = []
    for x in recommendation:
        relevance.append(1 if x in test else 0)
    k = len(relevance) if len(relevance)<k else k
    r = np.asarray(relevance)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

Upravené implementácie z [Ranking Metrics](https://gist.github.com/bwhite/3726239)

In [205]:
def test_precision(customer_id, cr_function):
    def split_products(products):
        middle = (len(products)+1)//2
        return [set(products[:middle]), products[middle:]]
    customer_products = data[data.customer_id == customer_id].sort_values(by ='timestamp')
    if len(customer_products) == 1:
        raise Exception(customer_id + ' has just one product')
    product_ids = list(customer_products.product_id)
    train, test = split_products(product_ids)
    recommendation = cr_function(train, count=10)
    return precision_at_k(recommendation, test, k=10)

Na vygenerovenie odporúčania bola použitá tabuľka `events`

## Výsledky

In [12]:
customer_ids = sorted(list(events.customer_id.unique()), key=lambda k: random.random())

In [13]:
len(customer_ids)

2210171

In [207]:
p = []
number_tested = 100
for cid in customer_ids:
    try:
        precision = test_precision(cid, cr3)
        if not np.isnan(precision):
            p.append(precision)
    except:
        pass
    if len(p) >= number_tested:
        break
print('number of tested customers', len(p))
print('average precision', np.mean(p))

number of tested customers 0
average precision nan


## Zapracovat
* boostovanie datumu, field asi nepojde
* pridat/ubrat fieldy
* dalsie metriky

# vi-challenge kaggle

In [203]:
vi_challenge_uID = pd.read_csv('vi_challenge_uID.csv', header=None)
len(vi_challenge_uID)

1000

In [163]:
events[events.customer_id == '050849d4-2b54-481b-b32b-822709bc18fd']

,customer_id,timestamp,event_type,product_id,title,category_name,price
14113018,050849d4-2b54-481b-b32b-822709bc18fd,2019-08-30 06:44:57.490581 UTC,view_item,023a7ed6-5dea-4915-a793-64c5e8b06eff,Trendyol Green Strapless Dress,Women>Women's Clothing>Dresses,38.46


In [197]:
cr3(set(events[events.customer_id == '0036fbad-4a20-47f7-a682-aebb275456dc'].product_id))

['24e8d758-f36e-4a46-a3f7-f580c9d8c5e8',
 '3a246c67-751d-4082-9d2c-903620112a64',
 '29d63b9b-c948-4df6-a76a-9ff5e5922003',
 'daeac09b-6a06-4abd-8ef8-83d9cd3f0bbb',
 '91a9de35-14c3-4623-b516-24d5fe4c7759',
 '23f6a87c-54cb-4ceb-b35f-32ee29903af6',
 'a1157781-f2c3-4466-827f-da713727a802',
 '65b07a7a-75dc-4e27-851d-17f674ea1364',
 'b2e5b028-b452-40af-be72-9840e0fda6b0',
 '7598fc5e-3e2a-454b-b173-a52dd6e0f250']

In [198]:
cr3(set(events[events.customer_id == '050849d4-2b54-481b-b32b-822709bc18fd'].product_id))

['023a7ed6-5dea-4915-a793-64c5e8b06eff',
 'fcb993c5-ea43-4e88-a8a2-44908f7e8316',
 '8262b507-0713-4794-b004-26794d10fed2',
 'd2dde762-279a-4bf5-b645-78ef0828780a',
 '3371f29f-16cf-43ca-9be5-2a03a5001da0',
 '34825f1b-de28-40fd-b08b-75e02dab6b4a',
 '99bd86ca-bd24-4e90-b644-98aa2a121011',
 '66a324b4-87e9-4568-b9eb-35ddc42768e2',
 'eb48ef50-15f8-4884-ab1d-047e12b41710',
 'bb079987-9fb4-4f2d-b953-04805d166a6f']

In [95]:
most_popular_products = list(events.product_id.value_counts().index[:10])

In [200]:
output = list()
for i, uid in enumerate(vi_challenge_uID[0]):
    customer_products = set(events[events.customer_id == uid].product_id)
    if customer_products:
        recomended = cr3(customer_products)
        print(i, ": ", len(recomended))
        for r in recomended:
            output.append([uid, r])
#             print('\t', [uid, r])
    else:
        print(i, "MPP")
        for r in most_popular_products:
            output.append([uid, r])
#             print('\t', [uid, r])

0 :  10
1 :  10
2 MPP
3 MPP
4 :  10
5 MPP
6 :  10
7 MPP
8 :  10
9 :  10
10 MPP
11 :  10
12 :  10
13 :  10
14 :  10
15 :  10
16 :  10
17 MPP
18 MPP
19 MPP
20 MPP
21 :  10
22 :  10
23 :  10
24 :  10
25 :  10
26 :  10
27 :  10
28 MPP
29 :  10
30 :  10
31 :  10
32 :  10
33 MPP
34 :  10
35 :  10
36 MPP
37 :  10
38 MPP
39 :  10
40 :  10
41 :  10
42 :  10
43 :  10
44 :  10
45 :  10
46 :  10
47 :  10
48 :  10
49 :  10
50 :  10
51 :  10
52 MPP
53 :  10
54 :  10
55 :  10
56 :  10
57 :  10
58 :  10
59 :  10
60 :  10
61 :  10
62 :  10
63 :  10
64 :  10
65 :  10
66 :  10
67 :  10
68 :  10
69 :  10
70 :  10
71 :  10
72 :  10
73 :  10
74 :  10
75 :  10
76 :  10
77 MPP
78 :  10
79 :  10
80 :  10
81 :  10
82 :  10
83 :  10
84 MPP
85 MPP
86 :  10
87 :  10
88 MPP
89 :  10
90 :  10
91 :  10
92 :  10
93 :  10
94 :  10
95 :  10
96 :  10
97 :  10
98 :  10
99 :  10
100 :  10
101 :  10
102 :  10
103 :  10
104 :  10
105 :  10
106 :  10
107 :  10
108 :  10
109 :  10
110 :  10
111 :  10
112 :  10
113 :  10
114 : 

857 :  10
858 :  10
859 MPP
860 :  10
861 MPP
862 :  10
863 :  10
864 :  10
865 :  10
866 :  10
867 :  10
868 :  10
869 :  10
870 :  10
871 :  10
872 MPP
873 :  10
874 :  10
875 :  10
876 :  10
877 :  10
878 MPP
879 :  10
880 :  10
881 :  10
882 :  10
883 :  10
884 MPP
885 :  10
886 :  10
887 :  10
888 :  10
889 :  10
890 :  10
891 :  10
892 :  10
893 :  10
894 :  10
895 :  10
896 :  10
897 :  10
898 :  10
899 :  10
900 :  10
901 :  10
902 MPP
903 :  10
904 :  10
905 :  10
906 :  10
907 :  10
908 :  10
909 :  10
910 :  10
911 :  10
912 MPP
913 :  10
914 :  10
915 MPP
916 :  10
917 MPP
918 :  10
919 MPP
920 :  10
921 :  10
922 :  10
923 :  10
924 :  10
925 :  10
926 :  10
927 :  10
928 :  10
929 :  10
930 MPP
931 :  10
932 MPP
933 :  10
934 :  10
935 :  10
936 :  10
937 :  10
938 :  10
939 :  10
940 MPP
941 :  10
942 :  10
943 :  10
944 MPP
945 MPP
946 :  10
947 :  10
948 :  10
949 MPP
950 MPP
951 :  10
952 :  10
953 :  10
954 :  10
955 :  10
956 MPP
957 :  10
958 :  10
959 :  10
960 : 

In [202]:
submission = pd.DataFrame(output, columns=['customer_id', 'product_id'])
submission.to_csv('harnusekSubmission--.csv', index=False)
len(submission)

10000

# -------------------------------------------------------------------------------------------

In [15]:
customer_ids[:10]

['22a0be82-d2fb-421a-a2a7-de0499eaaaac',
 '25729ed4-1073-427c-a3bd-6993e49878de',
 'f08aa62b-40d0-4dd5-9087-ffeb4e27c180',
 'b51f15ae-5594-4b48-a224-2e09190fde0c',
 'a6b2cfe9-b8d4-4ee1-a650-3aee40a5c089',
 '8783d5f2-746f-4dd4-aa05-c2960aea211f',
 'b1854f8d-4706-425a-8385-b9c822f63ac6',
 'c695f445-058a-4768-9c1a-09fb2b52c4b3',
 '5603a51a-1e83-4262-888e-1cf7a98fa008',
 'b5eb85fb-1e8e-45c4-a8a2-b8addc27dc3d']

In [49]:
def split_products(products):
    """
    Returns train:set and test:list data
    """
    middle = (len(products)+1)//2
    return [set(products[:middle]), products[middle:]]

In [187]:
tested_customer = events[events.customer_id == '5603a51a-1e83-4262-888e-1cf7a98fa008'].sort_values(by ='timestamp')
tested_customer

,customer_id,timestamp,event_type,product_id,title,category_name,price
3624786,5603a51a-1e83-4262-888e-1cf7a98fa008,2019-07-18 09:46:51.367906 UTC,view_item,31ae211d-57a9-4bfa-b95f-40776af03c3f,Lee Cooper Marl Backpack,Accessories >Backpacks and Bags,13.32
3625083,5603a51a-1e83-4262-888e-1cf7a98fa008,2019-07-18 09:48:35.856906 UTC,view_item,cc21bf26-0ca7-463f-973f-1f644a53514e,No Fear Urban Backpack,Accessories >Backpacks and Bags>Urban Backpacks,11.98
3625243,5603a51a-1e83-4262-888e-1cf7a98fa008,2019-07-18 09:49:28.294906 UTC,view_item,e9d3a1e9-cc3c-4101-b3ef-21ae55875213,Lonsdale Sport Backpack,Accessories >Backpacks and Bags>Urban Backpacks,13.32
3625263,5603a51a-1e83-4262-888e-1cf7a98fa008,2019-07-18 09:49:34.658906 UTC,view_item,db149e6d-553f-4d2b-9755-84e5224011ab,Ocean Pacific Backpack,Accessories >Backpacks and Bags>Urban Backpacks,9.28
3625267,5603a51a-1e83-4262-888e-1cf7a98fa008,2019-07-18 09:49:36.594906 UTC,view_item,cc21bf26-0ca7-463f-973f-1f644a53514e,No Fear Urban Backpack,Accessories >Backpacks and Bags>Urban Backpacks,11.98
3625301,5603a51a-1e83-4262-888e-1cf7a98fa008,2019-07-18 09:49:44.554906 UTC,view_item,e05b6cc7-950c-4d87-adf0-52457c6bd70f,Slazenger Shoe Bag,Accessories >Backpacks and Bags>Urban Backpacks,10.29
3625659,5603a51a-1e83-4262-888e-1cf7a98fa008,2019-07-18 09:51:52.953906 UTC,view_item,8a4d4ccb-d481-4d2c-b948-12943cf159e8,Hot Tuna Galaxy Backpack,Backpacks and Bags>Accessories >School Backpacks,11.98
3625806,5603a51a-1e83-4262-888e-1cf7a98fa008,2019-07-18 09:52:43.531906 UTC,view_item,a8282066-c887-40be-91e2-085ba96c059b,Hot Tuna Tuna Explorer B P 93,Backpacks and Bags>Accessories >School Backpacks,11.98
3626453,5603a51a-1e83-4262-888e-1cf7a98fa008,2019-07-18 09:56:14.835906 UTC,view_item,a535819f-7be0-4fad-84e5-f49229a81731,Hot Tuna Tuna Graph Suitcase84,Accessories >Backpacks and Bags>Travel bags an...,34.85


In [188]:
train, test = split_products(list(tested_customer.product_id))

In [189]:
train

{'31ae211d-57a9-4bfa-b95f-40776af03c3f',
 'cc21bf26-0ca7-463f-973f-1f644a53514e',
 'db149e6d-553f-4d2b-9755-84e5224011ab',
 'e9d3a1e9-cc3c-4101-b3ef-21ae55875213'}

In [190]:
test

['e05b6cc7-950c-4d87-adf0-52457c6bd70f',
 '8a4d4ccb-d481-4d2c-b948-12943cf159e8',
 'a8282066-c887-40be-91e2-085ba96c059b',
 'a535819f-7be0-4fad-84e5-f49229a81731']

In [191]:
cr2(train)

['db149e6d-553f-4d2b-9755-84e5224011ab',
 'cc21bf26-0ca7-463f-973f-1f644a53514e',
 'd7daceeb-0205-4bc8-879a-add2635c380c',
 'e9d3a1e9-cc3c-4101-b3ef-21ae55875213',
 '327be795-3882-47ac-a2d2-9c027f411eae',
 'c552587b-cb1e-4698-a101-3c8a4762b40a',
 '9480b05d-dd2f-4104-ac8a-ef3c1767a975',
 '731f8fcb-c9b2-430a-8a5a-f3597b579e58',
 '17990900-a833-4c08-92e1-4eb634269c51',
 '11b92fb0-5615-4201-bbfa-686fbf8daf11']